# Atividade 3

Faça tudo via dbt. 

Envia como resposta:
- A query SQL
- Comando utilizado via terminal no `dbt`
- Um print do resultado no próprio notebook, para isso é necessário criar uma pasta no repositório onde serão armazenadas as imagens, e para inserir no notebook é necessário criar uma célula `markdown` e inserir o seguinte comando: 

```markdown
![Nome da Foto](path/para/imagem)
```

1. Crie uma tabela `orders_by_year` a partir de `orders` que agrupa a quantidade de pedidos por ano, considerando a coluna `order_approved_at`

```
  {{ config(materialized='table') }}

SELECT 
  count (*) as order_count, 
  EXTRACT(
    YEAR 
    FROM 
      order_approved_at
  ) as year 
FROM 
  `dbt-shop-thiagol.dbt_shop.orders` 
group by 
  EXTRACT(
    YEAR 
    FROM 
      order_approved_at
  )
  ```

``dbt run --models shop.orders_by_year``



![exec1](../dados/atividade3/exec1.PNG)

2. Crie uma `view` que agrega os totais de pedidos por estado. Considere as tabelas _customers_ para obter os estados (coluna _customer_state_) e _orders_ para contabilizar a quantidade de pedidos.

```
  {{ config(materialized='view') }}
select 
  count(*) as total_pedido, 
  cust.customer_state as Estado 
from 
  `dbt_shop.customers` cust 
  inner join `dbt_shop.orders` order_itens on cust.customer_id = order_itens.customer_id 
group by 
  cust.customer_state
```

`` dbt run --models shop.total_order_state ``

![exec2](../dados/atividade3/exec2.PNG)

3. De acordo com o resultado do exercício 2 retorne apenas os estados que tem mais de 5000 pedidos, esse número deve ser passado via variável `number_of_orders` do `dbt`.

  ```
  {{ config(materialized='view') }}
SELECT 
  * 
FROM 
  `dbt-shop-thiagol.dataset_thiagolopes.total_order_state` 
where 
total_pedido > {{ var("number_of_orders") }}
```

``dbt run --models shop.query_number_of_orders --vars "{'number_of_orders': 5000}"``

![exec3](../dados/atividade3/exec3.PNG)

4. Crie uma `view` chamada `customers_by_region` que utiliza os dados da tabela `customers`. A visão deve realizar uma agregação por região do Brasil (Norte, Nordeste, Sudeste, Centro Oeste e Sul) e contar o número de clientes em cada uma delas.

  ```

SELECT COUNT(*) AS CUSTOMERS,
 CASE customer_state 
 WHEN 'RN' THEN 'NORDESTE'
 WHEN 'CE' THEN 'NORDESTE'
 WHEN 'RS' THEN 'SUL'
 WHEN 'SC' THEN 'SUL'
 WHEN 'SP' THEN 'SUDESTE'
 WHEN 'MG' THEN 'SUDESTE'
 WHEN 'BA' THEN 'NORDESTE'
 WHEN 'RJ' THEN 'SUDESTE'
 WHEN 'GO' THEN 'CENTRO OESTE'
 WHEN 'MA' THEN 'NORDESTE'
 WHEN 'PE' THEN 'NORDESTE'
 WHEN 'PB' THEN 'NORDESTE'
 WHEN 'ES' THEN 'SUDESTE'
 WHEN 'PR' THEN 'SUL'
 WHEN 'RO' THEN 'NORTE'
 WHEN 'MS' THEN 'CENTRO OESTE'
 WHEN 'PA' THEN 'NORTE'
 WHEN 'TO' THEN 'NORTE'
 WHEN 'MT' THEN 'CENTRO OESTE'
 WHEN 'PI' THEN 'NORTE'
 WHEN 'AL' THEN 'NORDESTE'
 WHEN 'AM' THEN 'NORTE'
 WHEN 'DF' THEN 'CENTRO OESTE'
 WHEN 'SE' THEN 'NORDESTE'
 WHEN 'RR' THEN 'NORTE'
 WHEN 'AP' THEN 'NORTE'
 WHEN 'AC' THEN 'NORTE' 
 END AS REGION  
FROM `dbt-shop-thiagol.dbt_shop.customers` 
GROUP BY REGION
  ```

``dbt run --models shop.customers_by_region``

![exec4](../dados/atividade3/exec4.PNG)

5. Crie uma tabela `items_costs` que contenha o preço total de um item (soma do valor do produto com o frete).

```
  {{ config(materialized='table') }}
select 
  iten.order_id, 
  iten.product_id, 
  iten.seller_id, 
  iten.order_item_id, 
  iten.shipping_limit_date, 
  (iten.price + iten.freight_value) as Iten_cost 
from 
  `dbt_shop.itens` iten
  ```

``dbt run --models shop.itens_cost``

![exec5](../dados/atividade3/exec5.PNG)

6. Crie uma `view` com o nome `orders_shipped_costs`, você deve calcular o preço total do pedido somado com o frete, apenas para pedidos enviado (shipped).

```
SELECT 
  ordem.order_id, 
  ordem.order_status, 
  sum(
    (iten.price + iten.freight_value)
  ) as iten_Cost 
FROM 
  `dbt-shop-thiagol.dbt_shop.orders` ordem 
  inner join `dbt-shop-thiagol.dbt_shop.itens` iten on iten.order_id = ordem.order_id 
group by 
  ordem.order_id, 
  ordem.order_status 
having 
  ordem.order_status = 'shipped'
  ```

``dbt run --models shop.orders_shipped_costs ``

![exec6](../dados/atividade3/exec6.PNG)